In [24]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np
from datetime import datetime

In [25]:
dateconv = lambda x: datetime.strptime(x, "%Y%m%d %H:%M:%S")

print "Date: ", dateconv("20160114  17:00:00")

Date:  2016-01-14 17:00:00


In [26]:
import os
print "Directory: ", os.listdir("../data/min/")

Directory:  ['ZS_201601_min.csv']


In [27]:
names=("date","open","high","low","close","volume","adjclose")
types=("|S18", float, float, float, float, int, float)

def readMinuteFile(minuteFileName):
    data = np.genfromtxt(minuteFileName, delimiter=",", skip_header=1, names=names, dtype=None, converters={'date': dateconv})
    #data = np.genfromtxt(minuteFileName, delimiter=",", skip_header=1, names=names, dtype=None)
    return data

m_data = readMinuteFile("../data/min/ZS_201601_min.csv")
print "m_data: ", m_data[0]
print "m_data[date]: ", m_data["date"][0:5]

#print "m_data: ", m_data
#print "m_data: ", np.asarray(m_data)



m_data:  (datetime.datetime(2016, 1, 14, 17, 0), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
m_data[date]:  [datetime.datetime(2016, 1, 14, 17, 0)
 datetime.datetime(2016, 1, 14, 17, 1)
 datetime.datetime(2016, 1, 14, 17, 2)
 datetime.datetime(2016, 1, 14, 17, 3)
 datetime.datetime(2016, 1, 14, 17, 4)]


In [30]:
t = datetime.timedelta(minutes = 10)
print "T: ", t

T:  0:10:00


In [43]:
def getDataForPeriod(m_data, lastMinuteProcessed, durationMinutes):
    nextIntervalEnd = lastMinuteProcessed + datetime.timedelta(minutes = durationMinutes)
    withinWindow = lambda x: x["date"] > lastMinuteProcessed and x["date"] < nextIntervalEnd
    return m_data[(m_data["date"] > lastMinuteProcessed) & (m_data["date"] <= nextIntervalEnd)]

lastMinuteProcessed = m_data["date"][0] - datetime.timedelta(seconds=1)
filteredData = getDataForPeriod(m_data, lastMinuteProcessed, 10)
print "Last Minute Processed: ", lastMinuteProcessed
print "Filtered Data:\n", filteredData

Last Minute Processed:  2016-01-14 16:59:59
Filtered Data:
[ (datetime.datetime(2016, 1, 14, 17, 0), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 1), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 2), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 3), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 4), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 5), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 6), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 7), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 8), 896.25, 896.25, 896.25, 896.25, 0, 896.25)
 (datetime.datetime(2016, 1, 14, 17, 9), 896.25, 896.25, 896.25, 896.25, 0, 896.25)]
